In [1]:
import os  
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import keras
from keras.datasets import cifar10
from keras import backend as K
from keras.layers import Input, Conv2D,GlobalAveragePooling2D, Dense, BatchNormalization, Activation
from keras.models import Model
from keras.layers import DepthwiseConv2D

from keras import optimizers,regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import he_normal
from keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint

num_classes        = 10
batch_size         = 64         # 64 or 32 or other
epochs             = 300
iterations         = 782       
USE_BN=True
DROPOUT=0.2 # keep 80%
CONCAT_AXIS=3
weight_decay=1e-4
DATA_FORMAT='channels_last' # Theano:'channels_first' Tensorflow:'channels_last'

log_filepath  = './mobilenet_n'

Using TensorFlow backend.


In [2]:
def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std  = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]
    return x_train, x_test

def scheduler(epoch):
    if epoch < 100:
        return 0.01
    if epoch < 200:
        return 0.001
    return 0.0001

# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test  = keras.utils.to_categorical(y_test, num_classes)
x_train, x_test = color_preprocessing(x_train, x_test)

In [4]:
def depthwise_separable(x,params):
    # f1/f2 filter size, s1 stride of conv
    '''
    (s1,f2) = params
    x = DepthwiseConv2D((3,3),strides=(s1[0],s1[0]), padding='same',depthwise_initializer="he_normal")(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = Conv2D(int(f2[0]), (1,1), strides=(1,1), padding='same',
               kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    '''
    (s1,f2)=params
    x=DepthwiseConv2D((3,3),strides=(s1[0],s1[0]),padding='same',depthwise_initializer='he_normal')(x)
    x=BatchNormalization(momentum=0.9,epsilon=1e-5)(x)
    x=Activation('relu')(x)
    x=Conv2D(int(f2[0]),(1,1),strides=(1,1),padding='same',kernel_initializer='he_normal',
             kernel_regularizer=regularizers.l2(weight_decay))(x)
    x=BatchNormalization(momentum=0.9,epsilon=1e-5)(x)
    x=Activation('relu')(x)
    return x

In [5]:
def MobileNet(img_input,shallow=False, classes=10):
    """Instantiates the MobileNet.Network has two hyper-parameters
        which are the width of network (controlled by alpha)
        and input size.
        # Arguments
            alpha: optional parameter of the network to change the 
                width of model.
            shallow: optional parameter for making network smaller.
            classes: optional number of classes to classify images
                into.
    """
    x = Conv2D(int(32), (3,3), strides=(2,2), padding='same',
              kernel_initializer="he_normal",kernel_regularizer=regularizers.l2(weight_decay))(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = depthwise_separable(x,params=[(1,),(64,)])
    x = depthwise_separable(x,params=[(2,),(128,)])
    x = depthwise_separable(x,params=[(1,),(128,)])
    x = depthwise_separable(x,params=[(2,),(256,)])
    x = depthwise_separable(x,params=[(1,),(256,)])
    x = depthwise_separable(x,params=[(2,),(512,)])
    if not shallow:
        for _ in range(5):
            x = depthwise_separable(x,params=[(1,),(512,)])
            
    x = depthwise_separable(x,params=[(2,),(1024,)])
    x = depthwise_separable(x,params=[(1,),(1024,)])

    x = GlobalAveragePooling2D()(x)
    out = Dense(classes, activation='softmax')(x)
    return out

In [6]:
img_input=Input(shape=(32,32,3))
output = MobileNet(img_input)
model=Model(img_input,output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 16, 16, 32)        0         
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 16, 16, 32)        320       
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 16, 16, 32)        0         
__________

In [6]:
# set optimizer
sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# set callback
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

# set data augmentation
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant',cval=0.)
datagen.fit(x_train)

# start training
model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                    steps_per_epoch=iterations,
                    epochs=epochs,
                    callbacks=cbks,
                    validation_data=(x_test, y_test))
model.save('mobilenet_n.h5')

Epoch 1/300
782/782 [==============================] - 53s 68ms/step - loss: 2.9389 - acc: 0.2943 - val_loss: 2.6469 - val_acc: 0.3918
Epoch 2/300
782/782 [==============================] - 46s 59ms/step - loss: 2.6100 - acc: 0.3984 - val_loss: 2.4728 - val_acc: 0.4355
Epoch 3/300
782/782 [==============================] - 46s 59ms/step - loss: 2.4676 - acc: 0.4403 - val_loss: 2.3028 - val_acc: 0.4897
Epoch 4/300
782/782 [==============================] - 45s 58ms/step - loss: 2.3823 - acc: 0.4685 - val_loss: 2.2578 - val_acc: 0.4977
Epoch 5/300
782/782 [==============================] - 46s 58ms/step - loss: 2.3207 - acc: 0.4884 - val_loss: 2.2103 - val_acc: 0.5087
Epoch 6/300
782/782 [==============================] - 46s 58ms/step - loss: 2.2445 - acc: 0.5094 - val_loss: 2.1421 - val_acc: 0.5376
Epoch 7/300
782/782 [==============================] - 44s 56ms/step - loss: 2.1730 - acc: 0.5256 - val_loss: 2.0495 - val_acc: 0.5598
Epoch 8/300
782/782 [==============================] - 

782/782 [==============================] - 40s 51ms/step - loss: 0.3808 - acc: 0.9543 - val_loss: 0.8724 - val_acc: 0.8399
Epoch 122/300
782/782 [==============================] - 41s 52ms/step - loss: 0.3753 - acc: 0.9568 - val_loss: 0.8725 - val_acc: 0.8394
Epoch 123/300
782/782 [==============================] - 39s 50ms/step - loss: 0.3727 - acc: 0.9564 - val_loss: 0.8790 - val_acc: 0.8414
Epoch 124/300
782/782 [==============================] - 39s 50ms/step - loss: 0.3695 - acc: 0.9586 - val_loss: 0.8957 - val_acc: 0.8386
Epoch 125/300
782/782 [==============================] - 38s 49ms/step - loss: 0.3742 - acc: 0.9561 - val_loss: 0.8868 - val_acc: 0.8410
Epoch 126/300
782/782 [==============================] - 39s 50ms/step - loss: 0.3678 - acc: 0.9586 - val_loss: 0.8910 - val_acc: 0.8439
Epoch 127/300
782/782 [==============================] - 39s 50ms/step - loss: 0.3665 - acc: 0.9578 - val_loss: 0.9031 - val_acc: 0.8400
Epoch 128/300
782/782 [==============================] 

Epoch 240/300
782/782 [==============================] - 39s 50ms/step - loss: 0.2392 - acc: 0.9881 - val_loss: 1.0477 - val_acc: 0.8404
Epoch 241/300
782/782 [==============================] - 39s 50ms/step - loss: 0.2384 - acc: 0.9878 - val_loss: 1.0575 - val_acc: 0.8390
Epoch 242/300
782/782 [==============================] - 39s 50ms/step - loss: 0.2390 - acc: 0.9883 - val_loss: 1.0665 - val_acc: 0.8389
Epoch 243/300
782/782 [==============================] - 39s 50ms/step - loss: 0.2382 - acc: 0.9882 - val_loss: 1.0735 - val_acc: 0.8387
Epoch 244/300
782/782 [==============================] - 38s 49ms/step - loss: 0.2363 - acc: 0.9888 - val_loss: 1.0512 - val_acc: 0.8415
Epoch 245/300
782/782 [==============================] - 39s 49ms/step - loss: 0.2386 - acc: 0.9883 - val_loss: 1.0658 - val_acc: 0.8393
Epoch 246/300
782/782 [==============================] - 39s 50ms/step - loss: 0.2378 - acc: 0.9883 - val_loss: 1.0488 - val_acc: 0.8423
Epoch 247/300
782/782 [==================

Epoch 300/300
782/782 [==============================] - 39s 49ms/step - loss: 0.2303 - acc: 0.9905 - val_loss: 1.0803 - val_acc: 0.8403
